# Global Configurations

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
cd drive/My Drive/Research/Word embedding usage for anomaly classification/anomaly predictions

/content/drive/My Drive/Research/Word embedding usage for anomaly classification/anomaly predictions


In [3]:
!ls

'anomaly predictions.npy'  'anomaly true values.npy'
 anomaly-pred-names.ipynb   names.txt


# Glove Configurations

In [22]:
!pip install annoy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 647 kB 7.3 MB/s 
  Created wheel for annoy: filename=annoy-1.17.1-cp38-cp38-linux_x86_64.whl size=394066 sha256=70828de139828578178c8c722d9f992503922ea11b4c1a0afdda80ebbbff9ffd
  Stored in directory: /root/.cache/pip/wheels/f9/93/19/30511c4a9ae6b4937455a134c34a39e13943e2c6f46fcd2ed2
Successfully built annoy


In [4]:
from torchtext.vocab import GloVe

glove_embedding = GloVe(name='6B', dim=300)

.vector_cache/glove.6B.zip: 862MB [02:39, 5.41MB/s]                           
100%|█████████▉| 399999/400000 [00:46<00:00, 8666.54it/s]


In [51]:
sample_emb = glove_embedding["heater"]
print(sample_emb)

tensor([ 5.8982e-01,  1.2853e-01, -2.5551e-01, -3.1889e-01, -8.7623e-02,
         1.2859e-02,  1.4171e-01, -1.0384e-02, -4.3160e-01, -1.9889e-01,
        -3.9704e-01,  1.6323e-01,  4.3911e-01, -8.0239e-02, -1.3800e-01,
         6.7812e-01, -1.9284e-01,  1.9053e-01,  1.2364e-01,  1.6491e-01,
         4.0839e-01,  2.7306e-01,  4.9132e-02, -3.5563e-01,  8.7836e-01,
        -2.9003e-02,  2.5714e-02,  2.9068e-01, -1.0572e-01, -1.4999e-01,
        -1.7801e-01,  3.1315e-01,  2.7408e-01, -6.0009e-02,  5.5913e-01,
         2.7362e-01, -3.3265e-01, -1.2201e-01,  2.6234e-01,  8.7877e-01,
         6.7768e-01,  6.2446e-01, -2.1079e-01, -2.9038e-01, -3.7958e-01,
         5.0669e-01,  2.6711e-01, -2.2382e-01, -4.6076e-01, -1.5841e-01,
         1.0319e-01,  1.1554e-01, -7.4071e-02, -3.0610e-01,  2.9418e-01,
         4.6540e-01,  8.8734e-01,  3.7541e-01,  4.0763e-01,  7.6471e-01,
        -1.3661e-01,  2.0058e-01,  5.1878e-01,  3.6418e-01, -4.0046e-01,
         5.5220e-01, -1.8583e-01, -2.1909e-01, -4.0

# Load Data

In [7]:
import numpy as np

y_pred = np.load("anomaly predictions.npy")
y_true = np.load("anomaly true values.npy")

actual_names = []
with open("names.txt", "r") as f:
  actual_names = f.readlines()


print(y_pred.shape)
print(y_true.shape)
print(len(actual_names))

(22000, 300)
(22000, 300)
22000


# Find Predictions

## find nearest word function

In [28]:
glove_keys = list(glove_embedding.stoi.keys())
print(glove_keys[100:110])

['so', 'them', 'what', 'him', 'united', 'during', 'before', 'may', 'since', 'many']


In [52]:
from annoy import AnnoyIndex
from tqdm import tqdm

num_trees = 50

ann_index = AnnoyIndex(300, 'euclidean')

for idx, key in tqdm(enumerate(glove_keys)):
  ann_index.add_item(idx, glove_embedding[key])


ann_index.build(num_trees)

400000it [04:23, 1519.80it/s]


True

In [53]:
def find_nearest_words(vector, k, ann_index, glove_embedding):
  closest_item_indices = ann_index.get_nns_by_vector(vector, k)
  result = []
  for idx in closest_item_indices:
    word = glove_embedding.itos[idx]
    result.append(word)
  

  return result


In [55]:
sample_vec = glove_embedding.get_vecs_by_tokens("espresso")

sample_result = find_nearest_words(sample_vec, 10, ann_index, glove_embedding)

print(sample_result)

['espresso', 'cappuccino', 'concoction', 'brewed', 'chocolate', 'iced', 'sipping', 'flavored', 'infused', 'dessert']


## find nearest words in my predictions

In [61]:
# find_nearest_words(vector, k, ann_index, glove_embedding)

knn = 1

final_result = {}

for idx in tqdm(range(len(actual_names))):
  true_name = actual_names[idx]
  if true_name not in final_result.keys():
    final_result[true_name] = {}
  
  # fix me
  pred_emb = y_pred[idx, :]
  pred_words = find_nearest_words(pred_emb, knn, ann_index, glove_embedding)

  for pred_name in pred_words:
    if pred_name not in final_result[true_name].keys():
      final_result[true_name][pred_name] = 1
    else:
      final_result[true_name][pred_name] += 1


100%|██████████| 22000/22000 [00:02<00:00, 10099.36it/s]


In [62]:
final_result

{'African elephant_Loxodonta africana\n': {'tvtonightnytsyn.com': 550},
 'apron\n': {'tvtonightnytsyn.com': 549, 'proverbial': 1},
 'bee\n': {'tvtonightnytsyn.com': 550},
 'bighorn_bighorn sheep_cimarron_Rocky Mountain bighorn_Rocky Mountain sheep_Ovis canadensis\n': {'tvtonightnytsyn.com': 550},
 'black stork_Ciconia nigra\n': {'tvtonightnytsyn.com': 550},
 'broom\n': {'tvtonightnytsyn.com': 548, 'proverbial': 2},
 'butcher shop_meat market\n': {'tvtonightnytsyn.com': 550},
 'centipede\n': {'tvtonightnytsyn.com': 550},
 'chimpanzee_chimp_Pan troglodytes\n': {'tvtonightnytsyn.com': 550},
 'confectionery_confectionary_candy store\n': {'tvtonightnytsyn.com': 550},
 'convertible\n': {'tvtonightnytsyn.com': 550},
 "dragonfly_darning needle_devil's darning needle_sewing needle_snake feeder_snake doctor_mosquito hawk_skeeter hawk\n": {'tvtonightnytsyn.com': 550},
 'espresso\n': {'tvtonightnytsyn.com': 550},
 'goldfish_Carassius auratus\n': {'tvtonightnytsyn.com': 549, 'proverbial': 1},
 'hou

In [63]:
import json

json = json.dumps(final_result)

with open("final_result_euclidean_{}.json".format(knn),"w") as f:
  f.write(json)
